
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# Demonstration - Monitoring Model Quality

In this demonstration, we will investigate how Databricks enables drift detection using various tools and features such as Notebooks, Queries, Dashboards, Alerts, while emphasizing Lakehouse Monitoring. Lakehouse Monitoring is leveraged to monitor the quality and statistical properties of data and ML models within the Databricks Lakehouse architecture. It allows you to track data quality, detect anomalies, and monitor the performance of ML models and model-serving endpoints.

As a part of the classroom setup, we will bring in a pre-generated inference table (with Change Data Feed enabled) utilizing Databricks' Marketplace as well as a baseline dataset to help demonstrate how we can detect and investigate drift. In practice, it is recommended that Delta Lake's Change Data Feed (CDF), which allows Databricks to track row-level changes between versions of a Delta table, be enabled on inference tables by default. By leveraging CDF, data scientists can more efficiently create and monitor inference tables. 



**Learning Objectives**

By the end of this demonstration, you will be able to do the following:

- Understand how to apply Lakehouse Monitoring for detecting different types of drift. Here are the core concepts and features:
    - Automated Dashboards built to analyze an inference profile
    - Know various ways in which you can locate the dashboard (Workspace,  and Lineage and Quality in Catalog Explorer)
    - Know various ways in which you can create additional widgets in the dashboard
    - Automate profile and drift metric tables
    - Out-of-the-box metrics and custom metrics
    - Inspect inference profile lineage and create alerts for sending notifications when metrics change
    - Using Lakehouse Monitoring to build custom for analyzing possible drift. 
        - Review of various types of drift and Lakehouse Monitoring can be applied
- Understand additional configurations and features using the UI
    - How to Update a baseline feature table
    - Update slicing configurations

**Note: This demonstration uses a mixture of SQL and Python.**

## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.
Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.
1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:
   - In the drop-down, select **More**.
   - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:
1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.
1. Find the triangle icon to the right of your compute cluster name and click it.
1. Wait a few minutes for the cluster to start.
1. Once the cluster is running, complete the steps above to select your cluster.


## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **17.3.x-cpu-ml-scala2.13**


* To utilize the generated Dashboard, you will need a SQL Warehouse.

## Classroom Setup

To get into the lesson, we first need to build some data assets and define some configuration variables required for this demonstration. When running the following cell, the output is hidden so our space isn't cluttered. To view the details of the output, you can hover over the next cell and click the eye icon. 

The cell after the setup, titled `View Setup Variables`, displays the various variables that were created. You can click the Catalog icon in the notebook space to the right to see that your catalog was created with no data.

In [0]:
%run ../Includes/Classroom-Setup-3.1

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

## Inference and Feature Tables

As a part of the classroom setup, we have created two tables:
1. `model_logs`: This is our inference table. It contains inference data from a sequence of queries that were made prior to this demo and will act as our dataset that is drifting away from the baseline data `(You can search for the dataset in the Marketplace by the name: Inference Dataset for Machine Learning Operations Associate Course.)`. 
1. `baseline_features`: This is our baseline table and acts as our source of truth.


<br>


**A Remark on Inference Tables and Inference Profiles**

It's important to distinguish between an inference table when thinking about the assets we will be working with. An inference table can be thought of as a collection of logs from the model that contain the input features at runtime and predicted labels. You can also include the true labels if they are available. On the other hand, an inference profile is an inference table that also contains profile metrics like F1-score, accuracy, or other evaluation metrics that measure the performance of the model on the inferred data.

In [0]:
primary_table_name = 'model_logs'
baseline_table_name = 'baseline_features'

primary_df = spark.read.format('delta').table(primary_table_name)
primary_pdf = primary_df.toPandas()

baseline_df = spark.read.format('delta').table(baseline_table_name)
baseline_pdf = baseline_df.toPandas()


inference_df = spark.read.format('delta').table('model_inference_table')



display(baseline_df)
display(primary_df)

### Motivating Drift Investigation
We'll take a cross-sectional snapshot of the distributions of BMI and Diabete_binary data to compare the ground truth to the data in our inference table.

In [0]:
# Compare distributions for a feature
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

#Randomly sample from the pandas dataframe baseline_pdf with samples equal to the size of primary_pdf
baseline_pdf_sample = baseline_pdf.sample(n=len(primary_pdf), random_state=1)


# KS test
stat, p_value = stats.ks_2samp(baseline_pdf_sample['BMI'], primary_pdf['BMI'])

# Visualization 1
plt.hist(baseline_pdf_sample['BMI'], bins=30, alpha=0.5, label="Training Data")
plt.hist(primary_pdf['BMI'], bins=30, alpha=0.5, label="Logged Data")
plt.legend()
plt.title("BMI Distribution: Training vs Logged (Drift Detected)")
plt.show()

# Visualization 2
plt.hist(baseline_pdf_sample['labeled_data'], bins=30, alpha=0.5, label="Training Data")
plt.hist(primary_pdf['Diabetes_binary'], bins=30, alpha=0.5, label="Predicted Data")
plt.legend()
plt.title("Target Distribution: Training vs Logged (Drift Detected)")
plt.show()

print(f'KS Test statistic value: {stat} and p-value: {p_value}')

## Lakehouse Monitoring

To monitor the performance of the model we used to build our premade `model_logs` table, we will attach the monitor to this table. Our baseline table `baseline_features` is what we can consider as our *good* or *source of truth* data. The Databricks monitor creates out-of-the-box metrics that are important and sets up two tables: 

1. Profile metrics table: This keeps track of profile metrics, which is a summary statistic computed for each column. 
1. Drift metrics table:  This keeps track of changes in profile metrics compared to baseline or previous time window. 

These two tables are used to automatically generate a monitoring dashboard. Later, we will uses these tables to build an alert for drift.

### Option 1: Using the UI

Let's first start with recalling how we can use the UI to setup with Lakehouse Monitoring

To add a monitor on the inference table, 

1. Open the **Catalog** menu from the left menu bar.

1. Select the table **model_logs** within your catalog and schema. 

1. Click on the **Quality** tab then on the **Enable** button.

1. After clicking Enable, select **Configure**.

1. As **Profile type** select **Inference profile**.

1. As **Problem type** select **classification**.

1. As the **Prediction column** select **Diabetes_binary**.

1. As the **Label column** select **labeled_data**

1. As **Metric granularities** select **5 minutes**, **1 hour**, and **1 day**. We will use the doctored timestamps to simulate requests that have been received over a large period of time. 

1. As **Timestamp column** select **timestamp**.

1. As **Model ID column** select **model_id**.

1. In Advanced Options --> **Unity catalog baseline table name (optional)** enter **Your baseline table name** from above along with your catalog and schema(Eg: catalog.schema.table_name).

1. Click the **Create** button.

### Option 2: Pythonic Approach
For a more programmatic approach, you can run the following code to initiate Lakehouse Monitoring. This method allows you to automate and customize the monitoring according to specific needs and thresholds.

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import MonitorInferenceLog, MonitorInferenceLogProblemType, MonitorInfoStatus, MonitorRefreshInfoState, MonitorMetric
w = WorkspaceClient()
full_primary_table_name = f'{DA.catalog_name}.{DA.schema_name}.{primary_table_name}'
full_baseline_table_name = f"{DA.catalog_name}.{DA.schema_name}.baseline_features"

In [0]:
help(w.quality_monitors.create)

In [0]:
#ML problem type, either "classification" or "regression"
PROBLEM_TYPE = MonitorInferenceLogProblemType.PROBLEM_TYPE_CLASSIFICATION

# Window sizes to analyze data over
GRANULARITIES = ["1 day"]   

# Directory to store generated dashboard
ASSETS_DIR = f"/Workspace/Users/{DA.username}/databricks_lakehouse_monitoring/{primary_table_name}"

# Optional parameters
SLICING_EXPRS = ["Age < 2", "Age > 15", "Sex = 1", "HighChol = 1"]   # Expressions to slice data with

In [0]:
print(f"Creating monitor for model_logs")
try: 
  info = w.quality_monitors.create(
    table_name=full_primary_table_name,
    inference_log=MonitorInferenceLog(
      timestamp_col='timestamp',
      granularities=GRANULARITIES,
      model_id_col='model_id',  # Model version number 
      prediction_col='Diabetes_binary',  # Ensure this column is of type DOUBLE
      problem_type=PROBLEM_TYPE,
      label_col='labeled_data'  # Ensure this column is of type DOUBLE
    ),
    baseline_table_name=full_baseline_table_name,
    slicing_exprs=SLICING_EXPRS,
    output_schema_name=f"{DA.catalog_name}.{DA.schema_name}",
    assets_dir=ASSETS_DIR
  )

  import time

  # Wait for monitor to be created
  while info.status ==  MonitorInfoStatus.MONITOR_STATUS_PENDING:
    info = w.quality_monitors.get(table_name=full_primary_table_name)
    time.sleep(10)

  assert info.status == MonitorInfoStatus.MONITOR_STATUS_ACTIVE, "Error creating monitor"
except Exception as e:
  print(f"Error creating monitor: {e}")

In [0]:
# A metric refresh will automatically be triggered on creation
refreshes = w.quality_monitors.list_refreshes(table_name=full_primary_table_name).refreshes
assert(len(refreshes) > 0)

run_info = refreshes[0]
while run_info.state in (MonitorRefreshInfoState.PENDING, MonitorRefreshInfoState.RUNNING):
  run_info = w.quality_monitors.get_refresh(table_name=full_primary_table_name, refresh_id=run_info.refresh_id)
  print(run_info)
  time.sleep(30)

assert run_info.state == MonitorRefreshInfoState.SUCCESS, "Monitor refresh failed"

In [0]:
w.quality_monitors.get(table_name=full_primary_table_name)

## Explore the Lakehouse Monitoring Dashboard

Methods for accessing the dashboard:
1. Run the next cell and click on the generated link. This will take you to the `model_logs` in the Catalog Explorer.
1. Select Quality and click on View Dashboard.  
1. The Dashboard (and other assets) are automatically registered to the lineage of `model_logs` as a part of comprehensive and automated tracking with Unity Catalog. 
1. Setting the `ASSETS_DIR` gives the location for where the generated dashboards are actually stored. In this demo, this is set to `f"/Workspace/Users/{DA.username}/databricks_lakehouse_monitoring/{primary_table_name}"`. 

The dashboard provides a comprehensive view of the data quality and statistical properties of the tables being monitored. It helps in tracking data integrity, statistical distribution, and data drift over time. This is where you will prebuilt widgets that incorporate the metrics generated from the profile and drift metric tables. The dashboard should be thought of as a starting point for monitoring for drift, hence it is fully customizable. 

- At the top left you can toggle between viewing the **Canvas** or the underlying **Data** that was generated with dashboard using SQL logic. Note that if a widget is currently being used, then the dataset created cannot be deleted. 
- Select widget that reads # Inferences on the left side of the canvas
    - Click on the kebab and select View Dataset. This will take you to the generated SQL code that you can use for further analysis with the SQL Editor or in this notebook.
    - Try editing the code and clicking Run to see how you can analyze the underlying tables generated as a part of the Dashboard creation.
- Back in the canvas, notice some metrics may appear as `null` or indicate **no data available**. This is expected since we are working with a sample dataset, since the automatically generated SQL code has logic that might not agree with this dataset such as comparing only baseline features or compute metrics on categorical features, which we are not considering here. If a baseline table was provided, drift metrics will be monitored. 
- For example, scroll down to the **Numerical Feature Drift** section and inspect the code for **# Features with Numerical Drift**. Notice that this SQL code only investigates the last window from the drift table. 


In [0]:
# Extract workspace URL
workspace_url = spark.conf.get('spark.databricks.workspaceUrl')

# Construct the monitor dashboard URL
monitor_dashboard_url = f"https://{workspace_url}/explore/data/{DA.catalog_name}/{DA.schema_name}/model_logs?o={DA.schema_name}&activeTab=quality"

print(f"Monitor Dashboard URL: {monitor_dashboard_url}")

**🚨 Instructor Note - Model Id Parameter Setup in in Data Tab of Dashboard**

If you see “Missing selection for parameter: Model Id” in the dashboard, this happens because the Model Id filter is not automatically applied to all widgets.

To fix this:

- In the dashboard toolbar, find the Model Id field at the top.
- Manually type * (asterisk) in the Model Id box - this selects all model IDs.
- Press Enter or click Run to refresh the dashboard.

💡 Typing `*` tells the dashboard to include every available Model Id, so all widgets and queries render correctly.


> **⚠️ Note:**  
> You may encounter a `CAST_INVALID_INPUT` error in the **Numerical Feature Quantile Drift** widget.  
> This occurs because the auto-generated query attempts to cast the string `"Baseline"` to a `TIMESTAMP` type in the `Window` column.  
> While this error does not affect other metrics or drift detection functionality, you can optionally resolve it using the steps below.

**Steps to Diagnose and Fix the Error (Optional)**
- **Step 1:**  
  Navigate to the **Numerical Feature Quantile Drift** widget in the monitoring dashboard.  
  If there's a rendering failure, you will see a `CAST_INVALID_INPUT` error message displayed within the widget.
- **Step 2:**  
  Click the **kebab menu (⋮)** in the top-right corner of the widget and select **View Dataset**.  
  This opens the SQL query that powers the visual.
- **Step 3:**  
  In the dataset editor, scroll down and click **Diagnose error**.  
  The Databricks Assistant will analyze the issue and provide a suggested fix.
- **Step 4:**  
  Click **Replace active cell content** to apply the Assistant’s suggested query.  
  The corrected query will ensure the `Window` column in the baseline portion uses a proper timestamp type, typically by replacing `"Baseline"` with `CAST(NULL AS TIMESTAMP)`.
- **Step 5:**  
  Click **Run** to execute the updated query.
- **Step 6:**  
  Once the query runs successfully, the widget should render correctly, and the error message will disappear.
This fix helps ensure type consistency in the `UNION` clause across datasets, resolving the rendering error in the visual.


## Create a New Visual

### Clone Method

- Click the kebab on the **Inference Load Over Time**
- Click **Clone**
- Click on the new visual
    - Notice the **Widget** pane has been automatically populated with the same configuration as the cloned visual.
- Under Visualization on the widget configuration menu to the right, select **Counter** from the dropdown menu. 
- Click on the dropdown menu for the Value and select **Count** under Field and select **SUM** under **Transform**
- Click on the dropdown menu under **Target** and select Window under Field and select **COUNT DISTINCT**
- Double click on the title to change the title to **# Inferences and Intervals Monitored**. 

You will see a new counter displayed now that shows the total inference load over time and how many distinct windows there are. You can also resize the visual by dragging the corner inward or outward. 

- Select the kebab again
- Here you can download the data for this visualization and copy a link to the widget if you would like to share it with your team. 

### Custom Visual

Additionally, you build your own visual.
- At the bottom of the canvas, there is a blue box that contains **Select** **Add a visualization**, **Add a textbox**, and **Add a filter**. Select **Add a visualization**. 
- In the visualization editor on the right of your screen:
        - Select **profile_over_time** under **Dataset**
        - Select **Counter** under **Visualization**
        - Select **f1_score** under **Field** and **COUNT** under **Transform**
        - Select **Title** under **Widget** and give it title **# F1-Scores Over Time**
- Finally, inspect the SQL generated for this widget. 

### Additional Customization

You can also build custom SQL code to build an even more customized visual by select **Create from SQL** at the very bottom of the Datasets pane to the left in the **Data** tab. Note this requires a detailed understanding of the profile and drift metrics tables and your specific goals for creating the new dataset(s).

## Explore the 2 Metric Tables

Now that we have inspected the automatically generated dashboard, let's take a look at the two Delta tables that built this dashboard. 

1. **Navigate to the Catalog Explorer**:
   - On the left-hand side of the Databricks workspace, click on **Catalog Explorer**.

1. **Locate Your Catalog**:
   - Find `dbacademy` catalog and select the `schema` that you are currently working in.

1. **Explore the New Tables**:
   - Look for two new tables:
     - `model_logs_drift_metrics`
     - `model_logs_profile_metrics`
   - Click on each table to explore the metrics that were calculated.

1. **Note on Metrics**:
   - Similar to the **Model Logs Dashboard**, some metrics may appear as `null` depending on the column's characteristics or data availability. For example, we have no categorical features within this dataset.

### A Closer Look at Out-Of-The-Box Metrics

The profile metrics table and drift metrics table created by Databricks Lakehouse Monitoring contain different types of information to help monitor and analyze data.

### Profile Metrics Table:
1. **Summary Statistics:**  
   Contains summary statistics for each column, such as count, number of null values, distinct count, minimum, maximum, average, standard deviation, and more.

2. **Time Window and Slices:**  
   Metrics are computed for specified time intervals (windows) and for each data slice defined by slicing expressions.

3. **Grouping Columns:**  
   Includes columns like time window, granularity, log type (input or baseline table), slice key and value, and model ID (for InferenceLog analysis).

4. **Model Accuracy Metrics:**  
   For InferenceLog analysis, it includes metrics like accuracy score, confusion matrix, precision, recall, and F1 score.

5. **Fairness and Bias Statistics:**  
   For classification models, it includes metrics like predictive parity, predictive equality, equal opportunity, and statistical parity.

### Drift Metrics Table:
1. **Drift Statistics:**  
   Tracks changes in distribution for a metric, comparing either to a previous time window (consecutive drift) or to a baseline distribution (baseline drift).

2. **Comparison Windows:**  
   Includes columns for the current and comparison time windows.

3. **Drift Types:**  
   Indicates whether the drift is compared to the previous window or a baseline table.

4. **Metrics Columns:**  
   Includes differences in count, average, percent null, percent zeros, percent distinct, and other statistical tests like:
   - Chi-squared test
   - KS test
   - Total variation distance
   - L-infinity distance
   - Jensen-Shannon distance
   - Wasserstein distance
   - Population stability index

In [0]:
# Display profile metrics table
profile_table = f"{DA.catalog_name}.{DA.schema_name}.model_logs_profile_metrics"
profile_df = spark.sql(f"SELECT * FROM {profile_table}")
display(profile_df.orderBy(F.rand()))

In [0]:
# Display profile metrics table
drift_table = f"{DA.catalog_name}.{DA.schema_name}.model_logs_drift_metrics"
drift_table_df = spark.sql(f"SELECT * FROM {drift_table}")
display(drift_table_df.orderBy(F.rand()))

Let's inspect the profile metrics table and the drift metrics table. Notice that the value of a cell will read `Null` when the metric does not apply.

### Example: Utilizing Profile Metrics

In [0]:
fb_cols = ["window", "model_id", "slice_key", "slice_value", "predictive_parity", "predictive_equality", "equal_opportunity", "statistical_parity"]
fb_metrics_df = profile_df.select(fb_cols).filter(f"column_name = ':table' AND slice_value = 'true'")
display(fb_metrics_df.orderBy(F.rand()).limit(10))

In [0]:
import pandas as pd
import json

# Load the profile metrics data from the Delta table
profile_table = f"{DA.catalog_name}.{DA.schema_name}.model_logs_profile_metrics"
profile_metrics_df = spark.read.table(profile_table)

# Convert to Pandas DataFrame
data = profile_metrics_df.toPandas()

# Extract and format 'window' column (start and end timestamps)
def process_window_column(row):
    if isinstance(row, dict):
        return {
            "start": pd.to_datetime(row.get("start", None)),
            "end": pd.to_datetime(row.get("end", None))
        }
    elif isinstance(row, str):  # If the row is still serialized as a string
        try:
            row_dict = json.loads(row)
            return process_window_column(row_dict)
        except Exception:
            return {"start": None, "end": None}
    else:
        return {"start": None, "end": None}

# Apply the processing function to the 'window' column
data['window'] = data['window'].apply(process_window_column)

# Extract start and end into separate columns for clarity
data['window_start'] = data['window'].apply(lambda x: x['start'])
data['window_end'] = data['window'].apply(lambda x: x['end'])

# Drop rows with null or empty F1-scores
data = data.dropna(subset=['f1_score'])
data = data.dropna(subset =['accuracy_score'])

# Extract relevant metrics
# Replace 'f1_score' with the actual column name for F1-score in your table
f1_scores = data[['window_start', 'window_end', 'f1_score', 'accuracy_score']]

# Display F1-scores over time
print("F1-scores over time (non-empty values):")
display(f1_scores)

### Example: Utilizing Drift Metrics

In [0]:
%sql
select * from model_logs_drift_metrics
where drift_type = 'BASELINE' AND slice_key is null and column_name = 'BMI'

In [0]:
import pandas as pd
import json

# Load the drift metrics data from the Delta table
drift_table = f"{DA.catalog_name}.{DA.schema_name}.model_logs_drift_metrics"
drift_metrics_df = spark.read.table(drift_table)

# Convert to Pandas DataFrame
data = drift_metrics_df.toPandas()

# Convert Timestamp objects to strings in 'window' and 'window_cmp'
def convert_timestamp_to_string(d):
    if isinstance(d, dict):
        for k, v in d.items():
            if isinstance(v, pd.Timestamp):
                d[k] = v.isoformat()
            elif isinstance(v, dict):
                d[k] = convert_timestamp_to_string(v)
    return d

data['window'] = data['window'].apply(convert_timestamp_to_string)
data['window_cmp'] = data['window_cmp'].apply(convert_timestamp_to_string)

# Ensure JSON fields are strings
data['window'] = data['window'].apply(json.dumps)
data['window_cmp'] = data['window_cmp'].apply(json.dumps)

# Convert the JSON string in 'window' and 'window_cmp' to dictionaries
for index, row in data.iterrows():
    row['window'] = json.loads(row['window'])
    row['window_cmp'] = json.loads(row['window_cmp'])

# Analyze the drift metrics
drift_thresholds = {
    "ks_test": 0.6,
}




def check_drift(row):
    ks_test_value = row['ks_test'].get('statistic') if isinstance(row['ks_test'], dict) else row['ks_test']
    if ks_test_value is not None and ks_test_value > drift_thresholds['ks_test']:
        return True
    return False

data['drift_detected'] = data.apply(check_drift, axis=1)

# Display rows with drift detected
drifted_rows = data[data['drift_detected']]
no_drifted_rows = data[~data['drift_detected']]

print("Rows with drift detected:")
display(drifted_rows)

print("\nRows with no drift detected:")
display(no_drifted_rows)


## Lineage and Alerts for Drift Detection

The following instructions are not meant to send you an alert to any device. It is only to demonstrate how to use lineage alongside alerts for your metric tables.

1. **Navigate to Queries**:
   - On the left-hand sidebar of the Databricks workspace, click on **Queries**.

1. **Create a Query**:
   - Click on **Create Query** in the top-right corner.
   - Write a query to calculate the maximum value of the median BMI value (this median value is computed per window):

         select
         max(median) as largest_median
         FROM model_logs_profile_metrics
         WHERE column_name = 'BMI'
         GROUP BY column_name;
   - Give the Query a name like `Drift Alert 1`
   > ****Note:**** Before running the query, make sure to set the current catalog and schema, and follow the `catalog.schema.table_name` structure.
   - Once the query is complete, click **Save** to save it.

1. **Create an Alert**:
   - Navigate to **Alerts** on the left-hand sidebar and click **Create Alert**.
   - Provide the alert with a **name** like `Test alert`.
   - In the SQL editor panel, paste the query you have saved before.
> ****Note:**** Before running the query, make sure to set the current catalog and schema, and follow the `catalog.schema.table_name` structure.

   - Click the **Run all** button to validate the query.
   - Define a **Trigger Condition**:
     - Example: Trigger the alert when the value is above 50 (50 is the max).
   - Click **Test condition**.
     - If the result satisfies the condition, you’ll see:`Triggered: Alert will trigger based on current data.`
   - Under **Notify**, search and add users or destinations. 
   - Finally, click **View alert** and Set the **Schedule**.
![alert](../Includes/images/alert.png)

1. **Monitor Lineage and Updates**:
   - Go back to the **Catalog** on the left sidebar.
   - Navigate to your **model_logs_profile_metrics** table.
   - Click on **Lineage** and then see the queries associated with this table.
      - You will see the `Drift Alert 1` listed there.
   - Whenever there’s an update or drift detected in your model or dataset, the alert will notify you automatically. You can then go to the table's **Lineage** tab to investigate the issue.

## Using Profile and Drift Metrics for Other Types of Drift
If we wish to monitor other types of drift (see below), then we can leverage the drift metrics table to do so.

Here we outline the metrics to monitor **concept drift**, **model quality drift**, and **bias drift**, along with guidance on their application.

| **Drift Type**      | **Definition**                                                                                                                                                                 | **Applicable Metrics**                                                                                                                                                                                                                                                                                        | **Additional Metrics**                                                                                                                                                                                                                              | **How to Use**                                                                                                                                                                                                                 |
|----------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Concept Drift**    | Changes in the relationship between input features and the target variable.                                                                                                 | - **KS Test (`ks_test`)**: Evaluate numeric feature drift.<br>- **Chi-Square Test (`chi_squared_test`)**: Analyze categorical feature drift.<br>- **TV Distance (`tv_distance`)**: Measure distributional differences.<br>- **Wasserstein Distance (`wasserstein_distance`)**: Capture numeric feature changes.<br>- **JS Distance (`js_distance`)**: Measure categorical similarity. | - **Predicted Probabilities (`pred_prob_delta`)**: Check prediction score changes.<br>- **Residual Distribution (`residual_delta`)**: Analyze residual distribution shifts over time.                                                               | - Compare feature distributions from training data to current data.<br>- Check conditional distributions of features given the target.<br>- Monitor predicted probabilities and residuals for deviations over time.                                   |
| **Model Quality Drift** | Degradation in the model's predictive performance over time.                                                                                                               | - **Count Delta (`count_delta`)**: Detect changes in data volume.<br>- **Avg Delta (`avg_delta`)**: Highlight feature mean shifts.<br>- **Percent Null Delta (`percent_null_delta`)**: Track changes in missing data rates.<br>- **Non-Null Columns Delta (`non_null_columns_delta`)**: Identify missing or new features. | - **Performance Metrics**: Monitor metrics like accuracy, RMSE, AUC, or F1 score.<br>- **Feature Importance Drift**: Observe changes in feature importance.<br>- **Residual Metrics**: Track residual trends (e.g., mean, variance).                  | - Monitor metrics for deviations from expected ranges.<br>- Use proxy metrics and performance metrics to assess drift.<br>- Address significant deviations with retraining or adjustments.                                                           |
| **Bias Drift**       | Changes that affect the fairness or equitable performance of the model across subgroups.                                                                                    | - **Chi-Square Test (`chi_squared_test`)**: Detect subgroup distributional shifts.<br>- **JS Distance (`js_distance`)**: Measure subgroup similarity changes.<br>- **Population Stability Index (`population_stability_index`)**: Track population shifts for subgroups.                                         | - **Outcome-Based Fairness Metrics**:<br> - **Disparate Impact**: Ratio of positive outcomes between subgroups.<br> - **Equalized Odds**: Differences in false positive/negative rates between subgroups.<br> - **Demographic Parity**: Positive prediction rates comparison.<br>- **Performance Metrics by Subgroup**: Assess subgroup-specific performance metrics (e.g., accuracy, recall). | - Monitor subgroup feature distributions and outcomes.<br>- Address fairness concerns with outcome-based metrics.<br>- Use metrics to identify and address bias comprehensively.                                                                    |

### **Overall Guidance**
- Use these metrics alongside domain knowledge and performance evaluations.
- Contextual interpretation is critical to avoid false positives or overcorrecting.
- Combine proxy metrics, qualitative checks, and direct performance metrics for a comprehensive drift monitoring framework.

## Additional Configurations

There are other types of advanced configurations you setup. Although, because this monitor was previously created, you will have to refresh the process by clicking on **Refresh Metrics** under **Quality** for the **model_logs** table. We provide a few below as a part of this demonstration.

### Custom Metric Creation: Aggregate metric (using the UI)

On top of the analysis and drift statistics that are automatically generated with Lakehouse Monitoring, you can create custom metrics as well. 

There are 3 types of custom metrics that can be used with Lakehouse Monitoring:
1. Aggregated metrics
1. Derived metrics
1. Drift metrics


To create and implement a custom metric in Databricks Lakehouse Monitoring, follow these steps:

1. **Access the Catalog**
   - Navigate to the **Catalog** from the left sidebar menu.

1. **Open Model Logs**
   - In the Catalog, locate and click on **model_logs**.

1. **Edit Monitor Configuration**
   - Click on the **Quality** tab.
   - Click **Configuration**.
   - Click on **Configuration** under Data profiling.

1. **Enable Advanced Options**
   - Scroll down to **Advanced Options** and expand the drop-down menu.

1. **Add Custom Metric**
   - Select **Add Custom Metric** from the menu.

1. **Define the Custom Metric**
   - Enter the following details for your custom metric:
     - **Name**: `AverageDifferenceFruitsVeggies`
     - **Type**: `Aggregate`
     - **Input Columns**: `Fruits`, `Veggies`
     - **Output Type**: `Double`
     - **Definition**:
       AVG(Fruits - Veggies)



1. **Update and Refresh Monitor**
   - Once the metric is added, update and refresh the monitor to compute and view the results.


An example of how to configure a custom metric this with Python (and SQL Jinja template) is provided here:

```
from databricks.sdk.service.catalog import MonitorMetric, MonitorMetricType
from pyspark.sql import types as T

custom_metric = MonitorMetric(
    type=MonitorMetricType.CUSTOM_METRIC_TYPE_AGGREGATE,
    name="avg_diff_f1_f2",
    input_columns=[":table"],
    definition="avg(Fruits - Veggies)",
    output_data_type=T.StructField("output", T.DoubleType()).json(),
)

```

To read more on a more programmatic approach, please read this [official documentation](https://docs.databricks.com/en/lakehouse-monitoring/custom-metrics.html).

## Conclusion

In this demonstration, we discussed Lakehouse Monitoring and took a deep dive into how to monitor various metrics using Dashboards. You learned about out-of-the-box metrics as well as how to create a custom metric with the UI and programmatically. We also discussed how **Lineage** with Delta tables and **Alerts** can be used within Databricks for sending updates for when drift does occur with your profile and drift metric tables. Finally, you learned about how to set advanced configurations with your inference table such as adding slicers and granularities.

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>